In [2]:
import numpy as np
import pandas as pd
from pandas import DataFrame,Series

In [3]:
df = DataFrame({'key1':['a','b','c','a','c'],
               'key2':['one','two','one','two','three'],
               'data1':np.random.randn(5),
               'data2':np.random.randn(5)})
df

,data1,data2,key1,key2
0,1.023629,2.087165,a,one
1,0.789319,0.728076,b,two
2,-0.982533,-0.319293,c,one
3,0.216595,-0.473751,a,two
4,-0.127350,0.349500,c,three


按 key1 进行分组并计算 data1 的平均值，实现方式很多，这里用： 访问 data1，并根据 key1 调用 groupby。

In [4]:
grouped = df['data1'].groupby(df['key1'])
grouped

变量 grouped 是一个 Groupby 对象，它实际上没有进行任何计算，但已经包含了执行运算所需的一切。

In [5]:
#计算平均值
grouped.mean()

key1
a    0.620112
b    0.789319
c   -0.554942
Name: data1, dtype: float64

如果一次传入多个数组，会得到如下结果：

In [9]:
means = df['data1'].groupby([df['key1'], df['key2']]).mean()
means

key1  key2 
a     one      1.023629
      two      0.216595
b     two      0.789319
c     one     -0.982533
      three   -0.127350
Name: data1, dtype: float64

In [10]:
means.unstack()

key2,one,three,two
key1,,,
a,1.023629,NaN,0.216595
b,NaN,NaN,0.789319
c,-0.982533,-0.12735,NaN


- 实际上分组键可以是任意长度适合的数组，如下：

In [13]:
import numpy as np
stats = np.array(['A','B','A','A','B'])
years = np.array([2005,2006,2006,2006,2005])
df['data1'].groupby([stats,years]).mean()

A  2005    1.023629
   2006   -0.382969
B  2005   -0.127350
   2006    0.789319
Name: data1, dtype: float64

此外，你还可以把列名用作分组键：

In [14]:
df.groupby('key1').mean()

,data1,data2
key1,,
a,0.620112,0.806707
b,0.789319,0.728076
c,-0.554942,0.015104


可以看到key2列没有，这是因为该列不是数值，俗称‘麻烦列’，所以从结果中排除了。

In [15]:
df.groupby(['key1','key2']).mean()

data1     data2
key1 key2                     
a    one    1.023629  2.087165
     two    0.216595 -0.473751
b    two    0.789319  0.728076
c    one   -0.982533 -0.319293
     three -0.127350  0.349500

- size方法，返回一个包含分组大小的Series。

In [16]:
df.groupby(['key1','key2']).size()

key1  key2 
a     one      1
      two      1
b     two      1
c     one      1
      three    1
dtype: int64

In [17]:
df.groupby('key1').size()

key1
a    2
b    1
c    2
dtype: int64

## 对分组进行迭代

In [18]:
for name,group in df.groupby('key1'):
    print(name)
    print(group)

a
      data1     data2 key1 key2
0  1.023629  2.087165    a  one
3  0.216595 -0.473751    a  two
b
      data1     data2 key1 key2
1  0.789319  0.728076    b  two
c
      data1     data2 key1   key2
2 -0.982533 -0.319293    c    one
4 -0.127350  0.349500    c  three


In [21]:
for (k1,k2),group in df.groupby(['key1','key2']):
    print(k1,k2)
    print(group)

a one
      data1     data2 key1 key2
0  1.023629  2.087165    a  one
a two
      data1     data2 key1 key2
3  0.216595 -0.473751    a  two
b two
      data1     data2 key1 key2
1  0.789319  0.728076    b  two
c one
      data1     data2 key1 key2
2 -0.982533 -0.319293    c  one
c three
     data1   data2 key1   key2
4 -0.12735  0.3495    c  three


- 有一个可能有用的运算，将数据片段做成一个字典：

In [22]:
pieces = dict(list(df.groupby('key1')))
pieces

{'a':       data1     data2 key1 key2
 0  1.023629  2.087165    a  one
 3  0.216595 -0.473751    a  two, 'b':       data1     data2 key1 key2
 1  0.789319  0.728076    b  two, 'c':       data1     data2 key1   key2
 2 -0.982533 -0.319293    c    one
 4 -0.127350  0.349500    c  three}

In [23]:
pieces['a']

,data1,data2,key1,key2
0,1.023629,2.087165,a,one
3,0.216595,-0.473751,a,two


- groupby 默认是在axis=0上进行分组，也可以在其他任何轴上分组，下面根据dtype对列分组：

In [24]:
df.dtypes

data1    float64
data2    float64
key1      object
key2      object
dtype: object

In [25]:
grouped = df.groupby(df.dtypes,axis=1)

In [26]:
dict(list(grouped))

{dtype('float64'):       data1     data2
 0  1.023629  2.087165
 1  0.789319  0.728076
 2 -0.982533 -0.319293
 3  0.216595 -0.473751
 4 -0.127350  0.349500, dtype('O'):   key1   key2
 0    a    one
 1    b    two
 2    c    one
 3    a    two
 4    c  three}

- 语法糖

    选取部分列进行聚合，尤其对于大数据量，很可能只需对部分列进行聚合：

In [28]:
df.groupby('key1')['data1']
df.groupby('key1')[['data2']]

是以下代码的语法糖：

In [29]:
df['data1'].groupby(df['key1'])
df[['data2']].groupby(df['key1'])

## 通过字典或Series进行分组

In [30]:
people = DataFrame(np.random.randn(5,5),
                  columns=['a','b','c','d','e'],
                  index=['Joe', 'Steve', 'Wes', 'Jim', 'Tom'])


In [31]:
people.ix[2:3, ['b','c']]=np.nan
people

,a,b,c,d,e
Joe,-1.265614,0.086115,1.216777,0.715948,0.742495
Steve,1.385583,0.879546,0.328880,0.349437,0.286163
Wes,0.959472,NaN,NaN,-0.363566,2.373446
Jim,0.311447,0.324140,-0.935236,0.480040,0.395466
Tom,0.576234,0.272281,-0.271944,-0.192782,-0.208079


In [32]:
mapping = {'a':'red','b':'red','c':'blue', 'd':'blue', 'e':'red', 'f':'orange'}

In [33]:
by_column = people.groupby(mapping,axis=1)

In [34]:
by_column.sum()

,blue,red
Joe,1.932725,-0.437004
Steve,0.678317,2.551292
Wes,-0.363566,3.332918
Jim,-0.455196,1.031053
Tom,-0.464726,0.640435


In [35]:
by_column.count()

,blue,red
Joe,2,3
Steve,2,3
Wes,1,2
Jim,2,3
Tom,2,3


### 通过函数进行分组

In [41]:
people

,a,b,c,d,e
Joe,-1.265614,0.086115,1.216777,0.715948,0.742495
Steve,1.385583,0.879546,0.328880,0.349437,0.286163
Wes,0.959472,NaN,NaN,-0.363566,2.373446
Jim,0.311447,0.324140,-0.935236,0.480040,0.395466
Tom,0.576234,0.272281,-0.271944,-0.192782,-0.208079


In [42]:
#按名字长度分组
people.groupby(len).sum()

,a,b,c,d,e
3,0.581539,0.682536,0.009597,0.639640,3.303328
5,1.385583,0.879546,0.328880,0.349437,0.286163


## 数据聚合

In [43]:
df

,data1,data2,key1,key2
0,1.023629,2.087165,a,one
1,0.789319,0.728076,b,two
2,-0.982533,-0.319293,c,one
3,0.216595,-0.473751,a,two
4,-0.127350,0.349500,c,three


In [14]:
def peak_to_pead(arr):
    return arr.max() - arr.min()

In [45]:
grouped = df.groupby('key1')

In [46]:
grouped.agg(peak_to_pead)

,data1,data2
key1,,
a,0.807034,2.560915
b,0.000000,0.000000
c,0.855183,0.668793


**注意：自定义函数要比自带的慢得多**

## 面向列的多函数应用

In [2]:
import pandas as pd

In [3]:
tips = pd.read_csv('/sun/bigdata/data/tips.csv')

In [5]:
tips.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [6]:
tips['tip_pct'] = tips['tip']/tips['total_bill']
tips.head()

,total_bill,tip,sex,smoker,day,time,size,tip_pct
0,16.99,1.01,Female,No,Sun,Dinner,2,0.059447
1,10.34,1.66,Male,No,Sun,Dinner,3,0.160542
2,21.01,3.50,Male,No,Sun,Dinner,3,0.166587
3,23.68,3.31,Male,No,Sun,Dinner,2,0.139780
4,24.59,3.61,Female,No,Sun,Dinner,4,0.146808


In [7]:
grouped = tips.groupby(['sex','smoker'])

In [10]:
grouped_pct = grouped['tip_pct']

In [11]:
grouped_pct.mean()

sex     smoker
Female  No        0.156921
        Yes       0.182150
Male    No        0.160669
        Yes       0.152771
Name: tip_pct, dtype: float64

In [12]:
grouped_pct.agg('mean')

sex     smoker
Female  No        0.156921
        Yes       0.182150
Male    No        0.160669
        Yes       0.152771
Name: tip_pct, dtype: float64

In [15]:
#多函数
grouped_pct.agg(['mean','std', peak_to_pead])

mean       std  peak_to_pead
sex    smoker                                  
Female No      0.156921  0.036421      0.195876
       Yes     0.182150  0.071595      0.360233
Male   No      0.160669  0.041849      0.220186
       Yes     0.152771  0.090588      0.674707

In [16]:
#指定列名
grouped_pct.agg([('foo','mean'),('bar','std')])

foo       bar
sex    smoker                    
Female No      0.156921  0.036421
       Yes     0.182150  0.071595
Male   No      0.160669  0.041849
       Yes     0.152771  0.090588

In [18]:
#对指定列执行指定运算
functions = ['count', 'max', 'mean']
grouped['tip_pct','total_bill'].agg(functions)

tip_pct                     total_bill                  
                count       max      mean      count    max       mean
sex    smoker                                                         
Female No          54  0.252672  0.156921         54  35.83  18.105185
       Yes         33  0.416667  0.182150         33  44.30  17.977879
Male   No          97  0.291990  0.160669         97  48.33  19.791237
       Yes         60  0.710345  0.152771         60  50.81  22.284500

In [19]:
functions = [('foo','count'), ('bar','max')]  #这里也可以指定列名
grouped['tip_pct','total_bill'].agg(functions)

tip_pct           total_bill       
                  foo       bar        foo    bar
sex    smoker                                    
Female No          54  0.252672         54  35.83
       Yes         33  0.416667         33  44.30
Male   No          97  0.291990         97  48.33
       Yes         60  0.710345         60  50.81

- 对不同的列应用不同的函数

In [21]:
grouped.agg({'size':'max','tip':'sum'})

size     tip
sex    smoker              
Female No         6  149.77
       Yes        4   96.74
Male   No         6  302.00
       Yes        5  183.07

- 无索引

In [22]:
tips.groupby(['sex','smoker'], as_index=False).mean()

,sex,smoker,total_bill,tip,size,tip_pct
0,Female,No,18.105185,2.773519,2.592593,0.156921
1,Female,Yes,17.977879,2.931515,2.242424,0.182150
2,Male,No,19.791237,3.113402,2.711340,0.160669
3,Male,Yes,22.284500,3.051167,2.500000,0.152771


In [23]:
tips.groupby(['sex','smoker'], ).mean()

total_bill       tip      size   tip_pct
sex    smoker                                          
Female No       18.105185  2.773519  2.592593  0.156921
       Yes      17.977879  2.931515  2.242424  0.182150
Male   No       19.791237  3.113402  2.711340  0.160669
       Yes      22.284500  3.051167  2.500000  0.152771

### 分组级运算和转换

In [27]:
from pandas import DataFrame,Series
import numpy as np
people = DataFrame(np.random.randn(5,5),
                  columns=['a','b','c','d','e'],
                  index=['Joe', 'Steve', 'Wes', 'Jim', 'Tom'])

In [28]:
people

,a,b,c,d,e
Joe,0.976571,0.218140,0.033487,0.022447,-0.328645
Steve,-0.898253,-0.030619,-0.449794,2.422999,1.458055
Wes,-0.670133,-1.012886,0.330553,0.340931,-1.563332
Jim,0.699725,0.507666,0.092751,-0.062108,-0.267298
Tom,1.209889,-0.381051,1.899696,0.828658,-0.699097


In [29]:
key = ['one','two','one','two','one']
people.groupby(key).transform(np.mean)

,a,b,c,d,e
Joe,0.505442,-0.391932,0.754579,0.397345,-0.863692
Steve,-0.099264,0.238524,-0.178522,1.180446,0.595378
Wes,0.505442,-0.391932,0.754579,0.397345,-0.863692
Jim,-0.099264,0.238524,-0.178522,1.180446,0.595378
Tom,0.505442,-0.391932,0.754579,0.397345,-0.863692


In [30]:
people.groupby(key).mean()

,a,b,c,d,e
one,0.505442,-0.391932,0.754579,0.397345,-0.863692
two,-0.099264,0.238524,-0.178522,1.180446,0.595378


- 分组距平

In [31]:
def demean(arr):
    return arr - arr.mean()

In [32]:
demeaned = people.groupby(key).transform(demean)
demeaned

,a,b,c,d,e
Joe,0.471129,0.610072,-0.721091,-0.374898,0.535046
Steve,-0.798989,-0.269142,-0.271273,1.242554,0.862677
Wes,-1.175575,-0.620954,-0.424026,-0.056414,-0.699640
Jim,0.798989,0.269142,0.271273,-1.242554,-0.862677
Tom,0.704446,0.010882,1.145117,0.431313,0.164594


In [33]:
#检查距平之后均值是否为0
demeaned.groupby(key).mean()

,a,b,c,d,e
one,-7.401487e-17,1.850372e-17,-7.401487e-17,0.0,7.401487e-17
two,0.000000e+00,0.000000e+00,0.000000e+00,0.0,-5.551115e-17
